# Load data

In [ ]:
import pandas as pd

df = pd.read_excel('Otchet_Za_3_Mes (1).xlsx', sheet_name='Сообщения', skiprows=13)
df = df[df['Тип'] == 'Комментарий']

# Text preprocessing

In [ ]:
!pip install pymorphy2

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pymorphy2
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stop_words = set(stopwords.words("russian"))
lemmatizer = pymorphy2.MorphAnalyzer()

def preprocess(text):
    text = list(filter(str.isalpha, word_tokenize(str(text).lower(), language="russian")))
    text = list(lemmatizer.parse(word)[0].normal_form for word in text)
    text = list(word for word in text if word not in stop_words)
    return ' '.join(text)

In [ ]:
preprocess('Я люблю играть с друзьями!')

'любить играть друг'

In [ ]:
texts = df['Сообщение'].dropna().apply(lambda row: preprocess(row))
texts = texts[texts != '']

# Saving and loading preprocessed texts

In [ ]:
texts.to_csv('preprocessed.csv')  # сохранить тексты в файлик
# НЕ ЗАБУДЬ СКАЧАТЬ ФАЙЛИК СЕБЕ НА КОМП, ИНАЧЕ ОН МОЖЕТ УДАЛИТЬСЯ ИЗ КОЛАБА ЧЕРЕЗ ВРЕМЯ

In [ ]:
texts = pd.read_csv('preprocessed.csv', index_col=0)['Сообщение']  # загрузить тексты из указанного файлика

# Tf-Idf and dimensionality reduction

In [ ]:
import numpy as np
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer

tf_idf = TfidfVectorizer().fit_transform(texts)
X = TruncatedSVD(100).fit_transform(tf_idf)

# Clustering

In [ ]:
from sklearn.cluster import DBSCAN, AgglomerativeClustering, KMeans, SpectralClustering

# clusters = DBSCAN(eps=0.1, metric='cosine').fit_predict(X)
clusters = AgglomerativeClustering(n_clusters=50, metric='cosine', linkage='average').fit_predict(X)

In [ ]:
np.unique(clusters)  # посмотреть номера кластеров и их количество

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49])

In [ ]:
(clusters == 0).sum()  # количество текстов в 6 кластере

242

In [ ]:
texts[clusters == 55]  # посмотреть все тексты в 55 кластере 

In [ ]:
df.loc[texts[clusters == 4].index]['Сообщение']  # посмотреть сырые тексты, соответствующие 4-му кластеру

# Удаление кластера

In [ ]:
# Осторожно! Стирает часть данных. Запускайте код с умом!
texts = texts[clusters != 4]  # удаляет все тексты из 4-го кластера
# Теперь сохрани оставшиеся тексты в файлик (см. раздел saving preprocessed texts)
texts = texts[-pd.Series(clusters).isin([2, 3, 5])]

# Удаление стоп-слов

In [ ]:
# удалить сразу все стоп слова из файлика
from nltk.corpus import stopwords
import nltk

nltk.download('stopwords')
stop_words = set(stopwords.words("russian"))
with open('stopwords.txt', 'r', encoding='utf-8') as file:
    for line in file:
        stop_words.add(line.strip())
texts = texts.apply(lambda t: ' '.join(word for word in str(t).split() if word not in stop_words))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# удалить из текстов очередное стоп-слово
texts = texts.apply(lambda t: ' '.join(word for word in str(t).split() if word != 'новоестопслово'))

In [ ]:
# удаляем пустые тексты
texts = texts[texts != '']

# Выгрузить эксельку

In [ ]:
def clusters_to_excel(raw_texts, preprocessed_texts, clusters):
    iterables = [list(np.unique(clusters)), ['Сырые тексты', 'Обработанные тексты']]
    index = pd.MultiIndex.from_product(iterables, names=['Номер кластера', 'Представление текста'])
    biggest_cluster_size = pd.Series(clusters).value_counts().sort_values().iloc[-1]
    df = pd.DataFrame(index=pd.Index(range(biggest_cluster_size)), columns=index)
    
    for clust in np.unique(clusters):
        df[clust, 'Сырые тексты'] = raw_texts[clusters == clust].reset_index(drop=True)
        df[clust, 'Обработанные тексты'] = preprocessed_texts[clusters == clust].reset_index(drop=True)
    
    df.to_excel('Кластеры.xlsx')

In [ ]:
clusters_to_excel(df.loc[texts.index]['Сообщение'], texts, clusters)